# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from hw_config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [79]:
weather_path = "../../CSV_files/clean_weather_data.csv"

weather_df = pd.read_csv(weather_path, encoding = "utf-8")
weather_df = weather_df.fillna(0)
weather_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Highest Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,east london,ZA,-33.02,27.91,21.05,66.0,0.0,8.13,1.587256e+09
1,1,hualmay,PE,-11.10,-77.61,21.55,78.0,29.0,3.32,1.587256e+09
2,2,yellowknife,CA,62.46,-114.35,-2.00,92.0,90.0,4.60,1.587256e+09
3,3,albany,US,42.60,-73.97,6.11,72.0,76.0,5.15,1.587256e+09
4,4,touros,BR,-5.20,-35.46,26.00,94.0,75.0,1.00,1.587256e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
weight = weather_df["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity= 10, point_radius = 1)

fig = gmaps.figure()
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [81]:
best_weather_df = weather_df.loc[(weather_df["Highest Temperature"] > 10) | 
                                 (weather_df["Highest Temperature"] < 25)].dropna()
best_weather_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Highest Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,east london,ZA,-33.02,27.91,21.05,66.0,0.0,8.13,1.587256e+09
1,1,hualmay,PE,-11.10,-77.61,21.55,78.0,29.0,3.32,1.587256e+09
2,2,yellowknife,CA,62.46,-114.35,-2.00,92.0,90.0,4.60,1.587256e+09
3,3,albany,US,42.60,-73.97,6.11,72.0,76.0,5.15,1.587256e+09
4,4,touros,BR,-5.20,-35.46,26.00,94.0,75.0,1.00,1.587256e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
hotel_df = best_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Highest Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,0,east london,ZA,-33.02,27.91,21.05,66.0,0.0,8.13,1.587256e+09,
1,1,hualmay,PE,-11.10,-77.61,21.55,78.0,29.0,3.32,1.587256e+09,
2,2,yellowknife,CA,62.46,-114.35,-2.00,92.0,90.0,4.60,1.587256e+09,
3,3,albany,US,42.60,-73.97,6.11,72.0,76.0,5.15,1.587256e+09,
4,4,touros,BR,-5.20,-35.46,26.00,94.0,75.0,1.00,1.587256e+09,


In [83]:
coordinates_df = hotel_df[["Latitude", "Longitude"]]
coordinates_df.head()

,Latitude,Longitude
0,-33.02,27.91
1,-11.10,-77.61
2,62.46,-114.35
3,42.60,-73.97
4,-5.20,-35.46


In [84]:
for index, row in coordinates_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    target_coordinates = (latitude, longitude)
    target_search = "hotel"
    target_radius = 5000 
    target_type = "hotel"

In [96]:
for index, row in coordinates_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    target_coordinates = f"{latitude}, {longitude}"
    target_search = "hotel"
    target_radius = 5000 
    target_type = "hotel"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params = params)
    hotel_data = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "NaN"

In [95]:
hotel_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Highest Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,0,east london,ZA,-33.02,27.91,21.05,66.0,0.0,8.13,1.587256e+09,Blue Lagoon Hotel
1,1,hualmay,PE,-11.10,-77.61,21.55,78.0,29.0,3.32,1.587256e+09,La Posada De Santa Maria
2,2,yellowknife,CA,62.46,-114.35,-2.00,92.0,90.0,4.60,1.587256e+09,The Explorer Hotel
3,3,albany,US,42.60,-73.97,6.11,72.0,76.0,5.15,1.587256e+09,NaN
4,4,touros,BR,-5.20,-35.46,26.00,94.0,75.0,1.00,1.587256e+09,Pousada Amigos do Vento


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [99]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))